In [1]:
# if y = x**2
def dy_dx(x):
  return 2*x

In [2]:
dy_dx(3)

6

In [3]:
import torch

PyTorch constructs a dynamic computation graph during runtime.

Each operation on tensors creates a node in this graph, and the graph is used to compute gradients by backpropagation.

In [4]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [5]:
y = x**2
y

tensor(9., grad_fn=<PowBackward0>)

In [6]:
y.backward()

In [7]:
# calculate dy/dx at x=3
x.grad # as diff of x**2 is 2*x

tensor(6.)

In [8]:
import math

# y = x**2
# z = sin y = sin(x**2) SO,
# dz/dx = 2 x cos(x**2) 

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [9]:
dz_dx(4)

-7.661275842587077

In [10]:
# initialization
x = torch.tensor(4.0, requires_grad=True)
x

tensor(4., requires_grad=True)

In [11]:
y = x ** 2 # Value calculated && A computation graph node is created to store the relationship
y

tensor(16., grad_fn=<PowBackward0>)

In [12]:
z = torch.sin(y) # sin(16) Value calculated && Another computation graph node is created to store the relationship
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [13]:
z.backward() # computes the gradient way back to x using chain rule differentiation
# https://pytorch.org/tutorials/beginner/basics/autograd_tutorial.html#:~:text=The-,backward,-pass%20kicks%20off

In [14]:
x.grad    #dz/dx 

tensor(-7.6613)

In [15]:
y.grad    #dz/dy

C:\Users\sk\AppData\Local\Temp\ipykernel_12460\4013874637.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  y.grad    #dz/dy


### BCE Loss using autograd

In [16]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)
w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [17]:
# Binary Cross-Entropy Loss for scalar
# formula -> - y log(y') - (1-y) log(1-y')
def binary_cross_entropy_loss(prediction, target): # calculates the loss
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [18]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability
y_pred

tensor(0.9988)

In [19]:
# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012)

In [20]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [21]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [22]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [23]:
# as we need dL wrt dw and db 
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [24]:
w

tensor(1., requires_grad=True)

In [25]:
b

tensor(0., requires_grad=True)

In [26]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [27]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [28]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [29]:
loss.backward()

In [31]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


In [32]:
# Lets check with this value -> ..


## Torch 

In [33]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [34]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [35]:
y.backward()

In [36]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [37]:
# clearing grad - to prevent grad accumulation
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [38]:
y = x ** 2 # repeat from here
y

tensor(4., grad_fn=<PowBackward0>)

In [39]:
y.backward()

In [40]:
x.grad

tensor(4.)

In [41]:
x.grad.zero_()

tensor(0.)

In [42]:
# disable gradient tracking
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [43]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [44]:
y.backward()

In [45]:
x.grad

tensor(4.)

## Turn off gradient tracking

### 1st Option

In [ ]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

In [46]:
x.requires_grad_(False)

tensor(2.)

In [47]:
x

tensor(2.)

In [48]:
y = x ** 2

In [49]:
y

tensor(4.)

In [50]:
try:
    y.backward()
except:
    print("RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn


### 2nd Option :- alternate and better way - use detach() if backward not required

In [51]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [53]:
z = x.detach()
z

tensor(2.)

In [54]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [55]:
y1 = z ** 2
y1

tensor(4.)

In [56]:
y.backward()
x.grad

In [57]:
try:
    y1.backward() # error
    z.grad
except:
    print("RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn


In [59]:
z.grad

In [60]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [61]:
y = x ** 2

In [62]:
y

tensor(4., grad_fn=<PowBackward0>)

In [63]:
y.backward()

In [64]:
x.grad

tensor(4.)

### 3rd Option :- (easiest)

In [69]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [70]:
with torch.no_grad():
    y = x**2

In [72]:
y # gradient tarcking is of 

tensor(4.)